```
From: https://github.com/ksatola
Version: 0.0.1

TODOs
1. 

```

# cnvrg CORE with Minikube on WSL_Ubuntu

- https://minikube.sigs.k8s.io/docs/start/
- https://app.cnvrg.io/docs/guides/minikube.html#prerequisites-prepare-your-local-environment
- https://helm.sh/docs/intro/install/
- local deployment: https://app.cnvrg.io/docs/guides/helm-light-install.html
- this is not for local: https://app.cnvrg.io/docs/guides/helm-install.html



K8s networking
- https://www.digitalocean.com/community/tutorials/how-to-inspect-kubernetes-networking
    - kubectl get pod -o wide --all-namespaces
    - kubectl get service --all-namespaces

# Install Minikube
```
#curl -LO https://storage.googleapis.com/minikube/releases/v1.16.0/minikube-linux-amd64
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube

# Test
minikube status
minikube start
kubectl get po -A

minikube dashboard

kubectl create deployment hello-minikube --image=k8s.gcr.io/echoserver:1.4
kubectl expose deployment hello-minikube --type=NodePort --port=8080

kubectl get services hello-minikube

# The easiest way to access this service is to let minikube launch a web browser for you
minikube service hello-minikube

# Your application is now available at http://localhost:7080/.
kubectl port-forward service/hello-minikube 7080:8080

minikube stop
minikube delete --all

```

## Setup Minikube Cluster
```
# Adjust available memory for Docker (not needed for WSL)

minikube config set EmbedCerts true

# Choose the right Kubernetes version 1.16.8 or earlier. 1.17 is not currently supported by cnvrg
minikube start --cpus=10 --memory='16g' --disk-size='30g' --kubernetes-version v1.16.8

minikube status

```

## Install Helm
https://helm.sh/docs/intro/install/
```
curl https://baltocdn.com/helm/signing.asc | sudo apt-key add -
sudo apt-get install apt-transport-https --yes
echo "deb https://baltocdn.com/helm/stable/debian/ all main" | sudo tee /etc/apt/sources.list.d/helm-stable-debian.list
sudo apt-get update
sudo apt-get install helm
```

## Deploy cnvrg CORE using Helm
```
helm repo add cnvrg https://charts.cnvrg.io
helm repo update

# Get node name
kubectl get nodes

# Deploy
helm install cnvrg cnvrg/cnvrg -n cnvrg --create-namespace \
--timeout 1500s  --wait \
--set global.external_ip=$(minikube ip) \
--set global.node=minikube \
--set global.high_availability=false

# https://stackoverflow.com/questions/59443834/helm-3-install-for-resouces-that-exist
# List all helm releases
helm list --all --all-namespaces
# helm uninstall <release-name> -n <namespace>

# Monitor deployment
watch kubectl -n cnvrg get pods

# Get the ip address of the cluster
kubectl -n cnvrg get svc | grep ingress | awk "{print $4}"

# Not working (you can find IP and port in Docker Desktop in the Container)
# I get a different URL than stated in Completing the Setup (https://app.cnvrg.io/docs/guides/helm-light-install.html)
http://127.0.0.1:30080/

# Check if we can connect to a k8s cluster
$ kubectl get nodes

$ helm repo add cnvrg https://charts.cnvrg.io
$ helm repo update


# Get K8s cluster IP
$ kubectl describe node docker-desktop
# 192.168.65.4

```

## Result (but cannot access to app)
```
$ helm install cnvrg cnvrg/cnvrg -n cnvrg --create-namespace \
> --timeout 1500s  --wait \
> --set global.external_ip=$(minikube ip) \
> --set global.node=minikube \
> --set global.high_availability=false
NAME: cnvrg
LAST DEPLOYED: Thu Nov  4 21:26:57 2021
NAMESPACE: cnvrg
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
🚀 Thank you for installing cnvrg.io!

Your installation of cnvrg.io is now in progress.
Wait until all the pods in the cnvrg namespace will be ready (kubectl get pods -n cnvrg)
and access to the app via: http://app.
Join our community at https://community.cnvrg.io
Talk to our team via email at hi@cnvrg.io

---------------------------------------------------------

$watch kubectl -n cnvrg get pods
Every 2.0s: kubectl -n cnvrg get pods                   ML: Thu Nov  4 21:34:37 2021
NAME                                           READY   STATUS      RESTARTS   AGE
app-799b99845b-l7kgc                           0/1     Init:0/4    0          6m27s
cnvrg-operator-6cf4964969-tf5xm                1/1     Running     0          7m40s
elastalert-c5fd97cf8-ghgnh                     1/1     Running     4          6m38s
elasticsearch-0                                1/1     Running     0          6m39s
fluentd-7t8t9                                  1/1     Running     0          6m37s
grafana-84cdf796dc-r6bp5                       1/1     Running     0          6m29s
hyper-7fcc7f597d-hw2t5                         1/1     Running     0          6m27s
istio-ingressgateway-8594dc7496-dhkcb          1/1     Running     0          6m45s
istio-operator-5bcc48b967-l44hm                1/1     Running     0          7m18s
istiod-8dbd659fb-7wdvn                         1/1     Running     0          6m57s
kibana-58f5fd99c6-xdqds                        1/1     Running     0          6m38s
kube-state-metrics-77664cb658-8p69r            3/3     Running     0          6m29s
minio-79b78959f4-cz64g                         1/1     Running     0          6m42s
minio-exporter-token-skgxn                     0/1     Completed   0          6m29s
mpi-operator-5b9c6b69fd-n9lmx                  1/1     Running     0          6m24s
node-exporter-tkgqk                            2/2     Running     0          6m30s
postgres-576f4d6667-xcln7                      1/1     Running     0          6m45s
prometheus-cnvrg-0                             3/3     Running     1          4m38s
prometheus-operator-57fb7dd7ff-hzkrg           2/2     Running     0          6m35s
redis-6fc69d7c46-qchv9                         1/1     Running     0          6m41s
searchkiq-55b79bd78b-m5plj                     0/1     Init:0/1    0          6m27s
sidekiq-fcc48477-4psx9                         0/1     Init:0/1    0          6m27s
sidekiq-fcc48477-nrwgl                         0/1     Init:0/1    0          6m27s
sidekiq-prometheus-exporter-7cb9dc8697-lvwjl   1/1     Running     0          6m29s
systemkiq-6648868845-d2d9z                     0/1     Init:0/1    0          6m27s

---------------------------------------------------------

$ helm list --all --all-namespaces
NAME    NAMESPACE       REVISION        UPDATED                                 STATUS          CHART           APP VERSION
cnvrg   cnvrg           1               2021-11-04 21:26:57.3734572 +0100 CET   deployed        cnvrg-2.25.0    2.25.0  

---------------------------------------------------------

$ kubectl -n cnvrg get svc | grep ingress | awk "{print $4}"
istio-ingressgateway          LoadBalancer   10.107.254.46    <pending>     80:30351/TCP,443:32042/TCP              17m

---------------------------------------------------------

$ kubectl get service --all-namespaces
NAMESPACE     NAME                          TYPE           CLUSTER-IP       EXTERNAL-IP   PORT(S)                                 AGE
cnvrg         app                           ClusterIP      10.103.125.62    <none>        8080/TCP                                22m
cnvrg         dcgm-exporter                 ClusterIP      10.96.243.215    <none>        9400/TCP                                22m
cnvrg         elastalert                    ClusterIP      10.111.26.121    <none>        80/TCP                                  22m
cnvrg         elasticsearch                 ClusterIP      10.104.39.89     <none>        9200/TCP                                22m
cnvrg         grafana                       ClusterIP      10.97.188.103    <none>        3000/TCP                                22m
cnvrg         hyper                         ClusterIP      10.109.170.13    <none>        5050/TCP                                22m
cnvrg         istio-ingressgateway          LoadBalancer   10.107.254.46    <pending>     80:30351/TCP,443:32042/TCP              22m
cnvrg         istio-operator                ClusterIP      10.111.239.90    <none>        8383/TCP                                23m
cnvrg         istiod                        ClusterIP      10.106.143.151   <none>        15010/TCP,15012/TCP,443/TCP,15014/TCP   23m
cnvrg         kibana                        ClusterIP      10.107.84.124    <none>        5601/TCP                                22m
cnvrg         kube-state-metrics            ClusterIP      None             <none>        8443/TCP,9443/TCP                       22m
cnvrg         minio                         ClusterIP      10.96.2.181      <none>        80/TCP                                  22m
cnvrg         node-exporter                 ClusterIP      None             <none>        9100/TCP                                22m
cnvrg         postgres                      ClusterIP      10.103.64.182    <none>        5432/TCP                                22m
cnvrg         prometheus                    ClusterIP      10.96.41.188     <none>        9090/TCP                                22m
cnvrg         prometheus-operated           ClusterIP      None             <none>        9090/TCP                                20m
cnvrg         prometheus-operator           ClusterIP      None             <none>        8443/TCP                                22m
cnvrg         redis                         ClusterIP      10.109.107.84    <none>        6379/TCP                                22m
cnvrg         sidekiq-prometheus-exporter   ClusterIP      None             <none>        80/TCP                                  22m
default       kubernetes                    ClusterIP      10.96.0.1        <none>        443/TCP                                 25m
kube-system   kube-dns                      ClusterIP      10.96.0.10       <none>        53/UDP,53/TCP,9153/TCP                  25m
kube-system   kubelet                       ClusterIP      None             <none>        10250/TCP,10255/TCP,4194/TCP            21m

---------------------------------------------------------

$ kubectl describe node minikube
Name:               minikube
Roles:              master
Labels:             beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/os=linux
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=minikube
                    kubernetes.io/os=linux
                    minikube.k8s.io/commit=0a0ad764652082477c00d51d2475284b5d39ceed
                    minikube.k8s.io/name=minikube
                    minikube.k8s.io/updated_at=2021_11_04T21_25_23_0700
                    minikube.k8s.io/version=v1.23.2
                    node-role.kubernetes.io/master=
Annotations:        kubeadm.alpha.kubernetes.io/cri-socket: /var/run/dockershim.sock
                    node.alpha.kubernetes.io/ttl: 0
                    volumes.kubernetes.io/controller-managed-attach-detach: true
CreationTimestamp:  Thu, 04 Nov 2021 21:25:20 +0100
Taints:             <none>
Unschedulable:      false
Lease:
  HolderIdentity:  minikube
  AcquireTime:     <unset>
  RenewTime:       Thu, 04 Nov 2021 21:57:21 +0100
Conditions:
  Type             Status  LastHeartbeatTime                 LastTransitionTime                Reason                       Message
  ----             ------  -----------------                 ------------------                ------                       -------
  MemoryPressure   False   Thu, 04 Nov 2021 21:56:56 +0100   Thu, 04 Nov 2021 21:25:19 +0100   KubeletHasSufficientMemory   kubelet has sufficient memory available
  DiskPressure     False   Thu, 04 Nov 2021 21:56:56 +0100   Thu, 04 Nov 2021 21:25:19 +0100   KubeletHasNoDiskPressure     kubelet has no disk pressure
  PIDPressure      False   Thu, 04 Nov 2021 21:56:56 +0100   Thu, 04 Nov 2021 21:25:19 +0100   KubeletHasSufficientPID      kubelet has sufficient PID available
  Ready            True    Thu, 04 Nov 2021 21:56:56 +0100   Thu, 04 Nov 2021 21:25:19 +0100   KubeletReady                 kubelet is posting ready status
Addresses:
  InternalIP:  192.168.49.2
  Hostname:    minikube
Capacity:
  cpu:                24
  ephemeral-storage:  263174212Ki
  hugepages-2Mi:      0
  memory:             105427376Ki
  pods:               110
Allocatable:
  cpu:                24
  ephemeral-storage:  263174212Ki
  hugepages-2Mi:      0
  memory:             105427376Ki
  pods:               110
System Info:
  Machine ID:                 8c119696cf5e4d94b6402a4e8e5af6fa
  System UUID:                8c119696cf5e4d94b6402a4e8e5af6fa
  Boot ID:                    769d16e7-d8fe-4579-bc4b-1e443a1238b4
  Kernel Version:             4.19.104-microsoft-standard
  OS Image:                   Ubuntu 20.04.2 LTS
  Operating System:           linux
  Architecture:               amd64
  Container Runtime Version:  docker://20.10.8
  Kubelet Version:            v1.16.8
  Kube-Proxy Version:         v1.16.8
PodCIDR:                      10.244.0.0/24
PodCIDRs:                     10.244.0.0/24
Non-terminated Pods:          (31 in total)
  Namespace                   Name                                            CPU Requests  CPU Limits  Memory Requests  Memory Limits  Age
  ---------                   ----                                            ------------  ----------  ---------------  -------------  ---
  cnvrg                       app-799b99845b-l7kgc                            2 (8%)        0 (0%)      4Gi (3%)         0 (0%)         29m
  cnvrg                       cnvrg-operator-6cf4964969-tf5xm                 0 (0%)        0 (0%)      0 (0%)           0 (0%)         30m
  cnvrg                       elastalert-c5fd97cf8-ghgnh                      100m (0%)     400m (1%)   200Mi (0%)       800Mi (0%)     29m
  cnvrg                       elasticsearch-0                                 1 (4%)        2 (8%)      1Gi (0%)         4Gi (3%)       29m
  cnvrg                       fluentd-7t8t9                                   200m (0%)     0 (0%)      300Mi (0%)       1000Mi (0%)    29m
  cnvrg                       grafana-84cdf796dc-r6bp5                        100m (0%)     200m (0%)   100Mi (0%)       200Mi (0%)     29m
  cnvrg                       hyper-7fcc7f597d-hw2t5                          100m (0%)     2 (8%)      200Mi (0%)       4Gi (3%)       29m
  cnvrg                       istio-ingressgateway-8594dc7496-dhkcb           100m (0%)     2 (8%)      128Mi (0%)       1Gi (0%)       29m
  cnvrg                       istio-operator-5bcc48b967-l44hm                 50m (0%)      200m (0%)   128Mi (0%)       256Mi (0%)     30m
  cnvrg                       istiod-8dbd659fb-7wdvn                          500m (2%)     0 (0%)      2Gi (1%)         0 (0%)         29m
  cnvrg                       kibana-58f5fd99c6-xdqds                         500m (2%)     1 (4%)      500Mi (0%)       2000Mi (1%)    29m
  cnvrg                       kube-state-metrics-77664cb658-8p69r             0 (0%)        0 (0%)      0 (0%)           0 (0%)         29m
  cnvrg                       minio-79b78959f4-cz64g                          1 (4%)        0 (0%)      2Gi (1%)         0 (0%)         29m
  cnvrg                       mpi-operator-5b9c6b69fd-n9lmx                   0 (0%)        0 (0%)      0 (0%)           0 (0%)         29m
  cnvrg                       node-exporter-tkgqk                             112m (0%)     270m (1%)   200Mi (0%)       220Mi (0%)     29m
  cnvrg                       postgres-576f4d6667-xcln7                       2 (8%)        0 (0%)      4Gi (3%)         0 (0%)         29m
  cnvrg                       prometheus-cnvrg-0                              1200m (5%)    200m (0%)   1074Mi (1%)      50Mi (0%)      27m
  cnvrg                       prometheus-operator-57fb7dd7ff-hzkrg            100m (0%)     200m (0%)   100Mi (0%)       200Mi (0%)     29m
  cnvrg                       redis-6fc69d7c46-qchv9                          500m (2%)     1 (4%)      1Gi (0%)         2Gi (1%)       29m
  cnvrg                       searchkiq-55b79bd78b-m5plj                      1 (4%)        0 (0%)      1Gi (0%)         0 (0%)         29m
  cnvrg                       sidekiq-fcc48477-4psx9                          1 (4%)        0 (0%)      4Gi (3%)         0 (0%)         29m
  cnvrg                       sidekiq-fcc48477-nrwgl                          1 (4%)        0 (0%)      4Gi (3%)         0 (0%)         29m
  cnvrg                       sidekiq-prometheus-exporter-7cb9dc8697-lvwjl    0 (0%)        0 (0%)      0 (0%)           0 (0%)         29m
  cnvrg                       systemkiq-6648868845-d2d9z                      500m (2%)     0 (0%)      500Mi (0%)       0 (0%)         29m
  kube-system                 coredns-5644d7b6d9-r4cgb                        100m (0%)     0 (0%)      70Mi (0%)        170Mi (0%)     31m
  kube-system                 etcd-minikube                                   0 (0%)        0 (0%)      0 (0%)           0 (0%)         30m
  kube-system                 kube-apiserver-minikube                         250m (1%)     0 (0%)      0 (0%)           0 (0%)         30m
  kube-system                 kube-controller-manager-minikube                200m (0%)     0 (0%)      0 (0%)           0 (0%)         30m
  kube-system                 kube-proxy-gngfg                                0 (0%)        0 (0%)      0 (0%)           0 (0%)         31m
  kube-system                 kube-scheduler-minikube                         100m (0%)     0 (0%)      0 (0%)           0 (0%)         30m
  kube-system                 storage-provisioner                             0 (0%)        0 (0%)      0 (0%)           0 (0%)         32m
Allocated resources:
  (Total limits may be over 100 percent, i.e., overcommitted.)
  Resource           Requests       Limits
  --------           --------       ------
  cpu                13712m (57%)   9470m (39%)
  memory             27052Mi (26%)  16160Mi (15%)
  ephemeral-storage  0 (0%)         0 (0%)
  hugepages-2Mi      0 (0%)         0 (0%)
Events:
  Type    Reason                   Age                From        Message
  ----    ------                   ----               ----        -------
  Normal  Starting                 32m                kubelet     Starting kubelet.
  Normal  NodeHasSufficientMemory  32m (x8 over 32m)  kubelet     Node minikube status is now: NodeHasSufficientMemory
  Normal  NodeHasNoDiskPressure    32m (x8 over 32m)  kubelet     Node minikube status is now: NodeHasNoDiskPressure
  Normal  NodeHasSufficientPID     32m (x7 over 32m)  kubelet     Node minikube status is now: NodeHasSufficientPID
  Normal  NodeAllocatableEnforced  32m                kubelet     Updated Node Allocatable limit across pods
  Normal  Starting                 31m                kube-proxy  Starting kube-proxy.

```

## Cleaning
```
# Upgrade cnvrg
helm upgrade cnvrg cnvrg/cnvrg

# Delete cnvrg
helm delete cnvrg 
kubectl delete namespace cnvrg

minikube stop

# Delete cluster
minikube delete --all
```